In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from skimage.segmentation import mark_boundaries
from skimage.util import montage2d as montage
from skimage.morphology import binary_opening, disk, label
import gc; gc.enable() # memory is tight
from keras import models, layers
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from PIL import Image
from scipy import ndimage
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.data import imread
import numpy as np
import matplotlib.pyplot as plt
import os
import time

Using TensorFlow backend.


In [2]:
sub_mono = pd.read_csv('F:\\Downloads\\all\\Submissions\\Submission_mono_ship.csv')

In [3]:
sub_simple_ships = pd.read_csv('F:\\Downloads\\all\\Submissions\\Submission_simple_list_resnet_e100.csv')

In [4]:
sub_harbors = pd.read_csv('F:\\Downloads\\all\\Submissions\\Submission_port_list_resnet_bysimple_e100.csv')

In [5]:
images = sub_harbors.ImageId
print(images.shape)
#print(list(set(images)))

(2124,)


## Найдем метрику под каждый класс изображений. На размеченной выборке теста проверим предсказание НС. 

### Функии декодирования местоположения пикселей из предсказанной маски

In [6]:
def multi_rle_encode(img, **kwargs):
    '''
    Encode connected regions as separated masks
    '''
    labels = label(img)
    if img.ndim > 2:
        return [rle_encode(np.sum(labels==k, axis=2), **kwargs) for k in np.unique(labels[labels>0])]
    else:
        return [rle_encode(labels==k, **kwargs) for k in np.unique(labels[labels>0])]

# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img, min_max_threshold=1e-3, max_mean_threshold=None):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    if np.max(img) < min_max_threshold:
        return '' ## no need to encode if it's all zeros
    if max_mean_threshold and np.mean(img) > max_mean_threshold:
        return '' ## ignore overfilled mask
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

def masks_as_image(in_mask_list):
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768), dtype = np.uint8)
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks = rle_decode(mask)
    return all_masks

def masks_as_color(in_mask_list):
    # Take the individual ship masks and create a color mask array for each ships
    all_masks = np.zeros((768, 768), dtype = np.float)
    scale = lambda x: (len(in_mask_list)+x+1) / (len(in_mask_list)*2) ## scale the heatmap image to shift 
    for i,mask in enumerate(in_mask_list):
        if isinstance(mask, str):
            all_masks[:,:] += scale(i) * rle_decode(mask)
    return all_masks

### Метрика

In [7]:
# Intersection over Union for Objects
def IoU(y_true, y_pred, tresh=1e-10):
    Intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    Union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - Intersection
    return K.mean( (Intersection + tresh) / (Union + tresh), axis=0)
# Intersection over Union for Background
def back_IoU(y_true, y_pred):
    return IoU(1-y_true, 1-y_pred)
# Loss function
def IoU_loss(in_gt, in_pred):
    #return 2 - back_IoU(in_gt, in_pred) - IoU(in_gt, in_pred)
    return 1 - IoU(in_gt, in_pred)

### Создадим y_true и y_pred

In [8]:
test_path = 'F:\\Downloads\\all\\test_v2'

In [9]:
pred_path = 'F:\\Downloads\\all\\Submissions\\final\\ResNet_simple_v3(5).csv'
pred_masks = pd.read_csv(pred_path)
print(len(list(set(pred_masks.ImageId))))
#pred_masks.head()

FileNotFoundError: File b'F:\\Downloads\\all\\Submissions\\final\\ResNet_simple_v3(5).csv' does not exist

In [ ]:
true_path = 'F:\\Downloads\\all\\Submissions\\test_etalon_fin.csv'
true_masks = pd.read_csv(true_path)
print(len(list(set(true_masks.ImageId))))
#true_masks.head()

In [13]:
import random
from multiprocessing import cpu_count
cpu_count = cpu_count()

import numpy as np
import pandas as pd
import cv2
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
%matplotlib inline


def mask_part(pic):
    '''
    Function that encodes mask for single ship from .csv entry into numpy matrix
    '''
    back = np.zeros(768**2)
    starts = pic.split()[0::2]
    lens = pic.split()[1::2]
    for i in range(len(lens)):
        back[(int(starts[i])-1):(int(starts[i])-1+int(lens[i]))] = 1
    return np.reshape(back, (768, 768, 1))

def is_empty(key, marks):
    '''
    Function that checks if there is a ship in image
    '''
    df = marks[marks['ImageId'] == key].iloc[:,1]
    if len(df) == 1 and type(df.iloc[0]) != str and np.isnan(df.iloc[0]):
        return True
    else:
        return False
    
def masks_all(key, marks):
    '''
    Merges together all the ship markers corresponding to a single image
    key = name of image
    df = csv which contains masks for key
    '''
    df = marks[marks['ImageId'] == key].iloc[:,1]
    masks= np.zeros((768,768,1))
    if is_empty(key, marks):
        return masks
    else:
        for i in range(len(df)):
            masks += mask_part(df.iloc[i])
        return np.transpose(masks, (1,0,2))

In [57]:
n = 0
score = 0
for c_img in list(set(pred_masks.ImageId)):
    true_mask = masks_all(c_img, true_masks)
    pred_mask = masks_all(c_img, pred_masks)
    intersection = np.sum(true_mask * pred_mask)
    union = (true_mask + pred_mask)
    union[union == 2] = 1
    union = np.sum(union)
    if union > 0:
        score += intersection / union
        n += 1
    
print('IoU = ', score / n)

IoU =  0.1269339797269488


In [21]:
n

5762

In [18]:
def iou(img_true, img_pred):
    i = np.sum((img_true*img_pred) >0)
    u = np.sum((img_true + img_pred) >0) + 0.0000000000000000001  # avoid division by zero
    return i/u

In [19]:
thresholds = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

def read_masks(img_name, df):
    mask_list = df.loc[df['ImageId'] == img_name, 'EncodedPixels'].tolist()
    all_masks = np.zeros((len(mask_list), 768,768))
    for idx, mask in enumerate(mask_list):
        if isinstance(mask, str):
            all_masks[idx] = rle_decode(mask)
    return all_masks

# original version of @raresbarbantan
def f2_prev(masks_true, masks_pred):
    # a correct prediction on no ships in image would have F2 of zero (according to formula),
    # but should be rewarded as 1
    if np.sum(masks_true) == np.sum(masks_pred) == 0:
        return 1.0
    
    f2_total = 0
    for t in thresholds:
        tp,fp,fn = 0,0,0
        ious = {}
        for i,mt in enumerate(masks_true):
            found_match = False
            for j,mp in enumerate(masks_pred):
                miou = iou(mt, mp)
                ious[100*i+j] = miou # save for later
                if miou >= t:
                    found_match = True
            if not found_match:
                fn += 1
                
        for j,mp in enumerate(masks_pred):
            found_match = False
            for i, mt in enumerate(masks_true):
                miou = ious[100*i+j]
                if miou >= t:
                    found_match = True
                    break
            if found_match:
                tp += 1
            else:
                fp += 1
        f2 = (5*tp)/(5*tp + 4*fn + fp)
        f2_total += f2
    
    return f2_total/len(thresholds)

def f2(masks_true, masks_pred):
    if np.sum(masks_true) == 0:
        return float(np.sum(masks_pred) == 0)
    
    ious = []
    mp_idx_found = []
    for mt in masks_true:
        for mp_idx, mp in enumerate(masks_pred):
            if mp_idx not in mp_idx_found:
                cur_iou = iou(mt,mp)
                if cur_iou > 0.5:
                    ious.append(cur_iou)
                    mp_idx_found.append(mp_idx)
                    break
    f2_total = 0
    for th in thresholds:
        tp = sum([iou > th for iou in ious])
        fn = len(masks_true) - tp
        fp = len(masks_pred) - tp
        f2_total += (5*tp)/(5*tp + 4*fn + fp)    

    return f2_total/len(thresholds)

In [23]:
pred_path = 'F:\\Downloads\\all\\FINAL_SUB\\sub_test_filt_01.csv'
pred_masks = pd.read_csv(pred_path)
true_path = 'F:\\Downloads\\all\\Submissions\\test_etalon_fin.csv'
true_masks = pd.read_csv(true_path)

In [24]:
sample_size = pred_masks.shape[0]
pred_files = list(pred_masks['ImageId'].unique())

In [25]:
print(sample_size)

def f2_from_fnames(fnames):
    score_sum = 0
    for fname in fnames:
        true_mask = read_masks(fname, true_masks)
        pred_mask = read_masks(fname, pred_masks)
        score_sum += f2(true_mask, pred_mask)
    return score_sum



n_workers = cpu_count
%time scores = Parallel(n_workers)(delayed(f2_from_fnames)(pred_files[i::n_workers]) for i in range(n_workers))
scores = sum(scores)
print(scores/sample_size)

17350
Wall time: 2min 29s
0.7314050061780952


In [97]:
29.2+7.7


36.9

In [96]:
1624/15606

0.10406254004869922

In [99]:
0.5+9.2*0.6874+3.6*0.778+36.9*0.737+30.1*1+11.9


78.82018000000001

In [17]:
317 + 101 + 1206 + 911

2535